#### GA WCFS ETL notebook

In [48]:
import pandas as pd

-- source

In [49]:
import pandas as pd

# Load the Excel file
file_path = "export.xlsx"  # Update path if needed
excel_file = pd.ExcelFile(file_path)

# Parse the Radiology sheet
radiology_df = excel_file.parse('Radiology')

general_medicine_df = excel_file.parse('General Medicine')

eval_mgmt_df = excel_file.parse('Evaluation and Management')

In [50]:
# Create a cleaned copy
radiology_cleaned = radiology_df.copy()

# Drop the unnamed initial columns (first 4 are empty)
radiology_cleaned = radiology_cleaned.drop(columns=radiology_cleaned.columns[:4])

# Rename columns for clarity
radiology_cleaned.columns = [
    "code", "modifier", "description", "prof_mar", "fud", "apc",
    "op_mar", "si", "asc_mar", "pi", "extra"
]

# Drop the last 'extra' column if it's all NaN
if radiology_cleaned['extra'].isna().all():
    radiology_cleaned = radiology_cleaned.drop(columns=['extra'])

# Drop rows where 'code' is missing
radiology_cleaned = radiology_cleaned[radiology_cleaned['code'].notna()]

# Convert MAR fields to numeric
mar_fields = ["prof_mar", "op_mar", "asc_mar"]
for field in mar_fields:
    radiology_cleaned[field] = pd.to_numeric(radiology_cleaned[field], errors='coerce')

# Preview cleaned data
display(radiology_cleaned.head())

# Optional: Save to CSV or Excel
# radiology_cleaned.to_csv("cleaned_radiology.csv", index=False)
# radiology_cleaned.to_excel("cleaned_radiology.xlsx", index=False)


,code,modifier,description,prof_mar,fud,apc,op_mar,si,asc_mar,pi,extra
0,70010,NaN,MYELOGRAPY POST FOSSA RS&I,169.82,XXX,5572.0,945.66,Q2,0.0,N1,NaN
1,70015,NaN,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,491.68,XXX,NaN,NaN,NaN,NaN,NaN,NaN
2,70015,26,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,164.88,XXX,NaN,NaN,NaN,NaN,NaN,NaN
3,70015,TC,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,326.80,XXX,5573.0,1968.84,Q2,0.0,N1,NaN
4,70030,NaN,RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,96.76,XXX,NaN,NaN,NaN,NaN,NaN,NaN


-- bring in dim_zip

In [51]:
dim_zip_df = pd.read_csv(r"C:\Users\ChristopherCato\OneDrive - clarity-dx.com\Fee Schedules\dim_zip.csv")

# Filter for Georgia ZIP codes
ga_zip_df = dim_zip_df[dim_zip_df['state'] == 'GA'].copy()

# Optional: Drop duplicates if needed
ga_zip_df = ga_zip_df.drop_duplicates(subset=['ZIP'])

# Preview Georgia ZIP mapping
display(ga_zip_df.head())

# Optional: Save for reference
# ga_zip_df.to_csv("ga_zip_mapping.csv", index=False)


,ZIP,COUNTY,county_name,city,state
14757,30002,13089,DeKalb County,AVONDALE ESTATES,GA
14758,30003,13135,Gwinnett County,NORCROSS,GA
14759,30004,13117,Forsyth County,ALPHARETTA,GA
14762,30005,13117,Forsyth County,ALPHARETTA,GA
14764,30006,13067,Cobb County,MARIETTA,GA


-- csv for db insertion

In [52]:
# Add 2 new columns for region mapping
radiology_cleaned['region_type'] = 'state'       # since GA has statewide rates
radiology_cleaned['region_value'] = 'GA'  
radiology_cleaned['rate_unit'] = 1
# Drop multiple columns
radiology_cleaned = radiology_cleaned.drop(columns=['fud', 'apc', 'pi', 'asc_mar', 'op_mar',
                                                    'si', 'extra'])        # value used to match in db

radiology_cleaned = radiology_cleaned.rename(columns={
    'prof_mar': 'rate',
    'code': 'proc_cd',
})

# Define your new column order
new_column_order = [
    'proc_cd', 'modifier', 'description',
    'rate', 'rate_unit',
    'region_type', 'region_value'
]

# Apply the new order
radiology_cleaned = radiology_cleaned[new_column_order]

# Save to CSV for import to database
output_path = "db_import_radiology.csv"
radiology_cleaned.to_csv(output_path, index=False)

display(radiology_cleaned.head())

,proc_cd,modifier,description,rate,rate_unit,region_type,region_value
0,70010,NaN,MYELOGRAPY POST FOSSA RS&I,169.82,1,state,GA
1,70015,NaN,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,491.68,1,state,GA
2,70015,26,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,164.88,1,state,GA
3,70015,TC,CISTERNOGRAPHY POSITIVE CONTRAST RS&I,326.80,1,state,GA
4,70030,NaN,RADIOLOGIC EXAMINATION EYE DETECT FOREIGN BODY,96.76,1,state,GA


-- general medicine

In [53]:


# Start with a cleaned copy of General Medicine
genmed_cleaned = general_medicine_df.copy()

# Drop first 4 unnamed empty columns
genmed_cleaned = genmed_cleaned.drop(columns=genmed_cleaned.columns[:4])

# Rename columns for consistency
genmed_cleaned.columns = [
    "code", "modifier", "description", "prof_mar", "fud", "apc",
    "op_mar", "si", "asc_mar", "pi", "extra"
]

# Drop 'extra' column if it's all NaN
if genmed_cleaned['extra'].isna().all():
    genmed_cleaned = genmed_cleaned.drop(columns=['extra'])

# Drop rows where 'code' is missing
genmed_cleaned = genmed_cleaned[genmed_cleaned['code'].notna()]

# Normalize code to string and remove decimals
genmed_cleaned['code'] = genmed_cleaned['code'].astype(int).astype(str)

# Add is_by_report column based on 'prof_mar' value
genmed_cleaned['is_by_report'] = genmed_cleaned['prof_mar'].str.upper() == 'BR'

# Set prof_mar to NaN where it's 'BR' to allow numeric conversion
genmed_cleaned.loc[genmed_cleaned['is_by_report'], 'prof_mar'] = None

# Convert rate columns to numeric
rate_fields = ['prof_mar', 'op_mar', 'asc_mar']
for field in rate_fields:
    genmed_cleaned[field] = pd.to_numeric(genmed_cleaned[field], errors='coerce')

# Add region columns
genmed_cleaned['region_type'] = 'state'
genmed_cleaned['region_value'] = 'GA'
genmed_cleaned['rate_unit'] = 1

genmed_cleaned = genmed_cleaned.rename(columns={
    'prof_mar': 'rate',
    'code': 'proc_cd',
})

# Define your new column order
final_columns = [
    'proc_cd', 'modifier', 'description',
    'rate', 'rate_unit', 'is_by_report',
    'region_type', 'region_value'
]

genmed_final = genmed_cleaned[final_columns]

# Export to CSV
output_genmed_path = "db_import_general_medicine.csv"
genmed_final.to_csv(output_genmed_path, index=False)

display(genmed_final.head())


,proc_cd,modifier,description,rate,rate_unit,is_by_report,region_type,region_value
0,90281,NaN,IMMUNE GLOBULIN IG HUMAN IM USE,NaN,1,True,state,GA
1,90283,NaN,IMMUNE GLOBULIN IGIV HUMAN IV USE,NaN,1,True,state,GA
2,90284,NaN,IMMUNE GLOBULIN HUMAN SUBQ INFUSION 100 MG EA,NaN,1,True,state,GA
3,90287,NaN,BOTULINUM ANTITOXIN EQUINE ANY ROUTE,NaN,1,True,state,GA
4,90288,NaN,BOTULISM IMMUNE GLOBULIN HUMAN INTRAVENOUS USE,NaN,1,True,state,GA


-- evaluation & management

In [56]:
# Start with a cleaned copy of Evaluation and Management
eval_mgmt_cleaned = eval_mgmt_df.copy()

# Drop first 4 unnamed empty columns
eval_mgmt_cleaned = eval_mgmt_cleaned.drop(columns=eval_mgmt_cleaned.columns[:4])

# Rename columns for consistency
eval_mgmt_cleaned.columns = [
    "code", "modifier", "description", "prof_mar", "fud", "apc",
    "op_mar", "si", "asc_mar", "pi"
]

# Drop rows where 'code' is missing
eval_mgmt_cleaned = eval_mgmt_cleaned[eval_mgmt_cleaned['code'].notna()]

# Normalize code to string and strip whitespace
eval_mgmt_cleaned['code'] = eval_mgmt_cleaned['code'].astype(str).str.strip()

# Add is_by_report column based on 'prof_mar' value
eval_mgmt_cleaned['is_by_report'] = eval_mgmt_cleaned['prof_mar'].str.upper() == 'BR'

# Set prof_mar to NaN where it's 'BR' to allow numeric conversion
eval_mgmt_cleaned.loc[eval_mgmt_cleaned['is_by_report'], 'prof_mar'] = None

# Convert rate columns to numeric
rate_fields = ['prof_mar', 'op_mar', 'asc_mar']
for field in rate_fields:
    eval_mgmt_cleaned[field] = pd.to_numeric(eval_mgmt_cleaned[field], errors='coerce')

# Add region and rate_unit columns
eval_mgmt_cleaned['region_type'] = 'state'
eval_mgmt_cleaned['region_value'] = 'GA'
eval_mgmt_cleaned['rate_unit'] = 1

# Rename columns to match db format
eval_mgmt_cleaned = eval_mgmt_cleaned.rename(columns={
    'prof_mar': 'rate',
    'code': 'proc_cd',
})

# Define final column order
final_columns = [
    'proc_cd', 'modifier', 'description',
    'rate', 'rate_unit', 'is_by_report',
    'region_type', 'region_value'
]

eval_mgmt_final = eval_mgmt_cleaned[final_columns]

# Export to CSV
output_evalmgmt_path = "db_import_evaluation_and_management.csv"
eval_mgmt_final.to_csv(output_evalmgmt_path, index=False)

display(eval_mgmt_final.head())


,proc_cd,modifier,description,rate,rate_unit,is_by_report,region_type,region_value
0,99091,NaN,COLLJ & INTERPJ PHYSIOL DATA MIN 30 MIN EA 30 D,94.75,1,False,state,GA
1,99202,NaN,OFFICE O/P NEW SF 15 MIN,127.70,1,False,state,GA
2,99203,NaN,OFFICE O/P NEW LOW 30 MIN,197.15,1,False,state,GA
3,99204,NaN,OFFICE O/P NEW MOD 45 MIN,295.43,1,False,state,GA
4,99205,NaN,OFFICE O/P NEW HI 60 MIN,389.59,1,False,state,GA
